In [2]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
from astropy.coordinates import EarthLocation
import astropy.units as u
# from icecream import ic, install

from astropaul.database import database_connection
import astropaul.targetlistcreator as tlc
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

# install() # icecream will now work in imported packages

%load_ext autoreload
%autoreload 2


In [3]:
html_dir = "../../../../Observing Files/APO Observing 2025-11-04"
html.clear_directory(html_dir)
session = tlc.ObservingSession(ap.Observer.at_site("APO"))

session.add_full_day("2025-11-04")
session.add_full_day("2025-11-05")
session.add_half_day("2025-11-06", first_half=True)

synthetic_phase_percent = 0.02

phase_event_defs = [
    ph.PhaseEventDef("Mid Eclipse", ph.calc_mid_eclipse),
    ph.PhaseEventDef("Eclipse", ph.calc_mid_eclipse),
    ph.PhaseEventDef(
        "Not in Eclipse", partial(ph.calc_time_of_gress, synthetic_phase_percent=synthetic_phase_percent, ingress=False)
    ),
    ph.PhaseEventDef("Eclipse", partial(ph.calc_time_of_gress, synthetic_phase_percent=synthetic_phase_percent, ingress=True)),
]

min_altitude = 35 * u.deg
max_magnitude = 13

with database_connection() as conn:
    name = "APO Observing 2025-11-04"
    creator = tlc.TargetListCreator(name=name, connection=conn, phase_event_defs=phase_event_defs)
    creator.steps = [
        tlc.add_targets,
        partial(tlc.filter_targets, criteria=lambda df: df["Target Type"].isin(["QuadEB", "SextEB"])),
        tlc.ancillary_data_from_tess,
        partial(tlc.filter_targets, criteria=lambda df: df["Vmag"] < max_magnitude),
        partial(
            tlc.add_observability,
            observing_session=session,
            calc_moon_distance=True,
            observability_threshold=(min_altitude, 80 * u.deg),
        ),
        partial(tlc.filter_targets, criteria=lambda df: df["Observable Any Night"]),
        partial(tlc.add_database_table, table_name="ephemerides"),
        partial(tlc.add_database_table, table_name="dssi_observations"),
        partial(
            tlc.add_system_configuration, table_name="DSSI Observations", time_column="Mid JD", eclipse_table="SIDE Observations"
        ),
        partial(tlc.add_database_table, table_name="speckle_detections"),
        partial(
            tlc.add_phase_events,
            observing_session=session,
            phase_event_defs=phase_event_defs,
            event_types=["Mid Eclipse", "Eclipse"],
        ),
        partial(tlc.add_database_table, table_name="pepsi_observations"),
        tlc.add_pepsi_evaluations,
    ]
    tl = creator.calculate(verbose=False)

# html.render_observing_pages(tl, None, {}, html_dir)
print(tl.summarize())

Name: APO Observing 2025-11-04
Criteria
    lambda df: df["Target Type"].isin(["QuadEB", "SextEB"])
    lambda df: df["Vmag"] < 13
    AltitudeConstraint: {'min': 35.0, 'max': 80.0, 'boolean_constraint': True}
    lambda df: df["Observable Any Night"]
97 targets:
      93 QuadEB
       4 SextEB
Column Count (primary, secondary):
    Target: (3, 4)
    TESS Data: (2, 123)
    Observable: (5, 12)
    Count: (5, 0)
Associated tables:
       3 rows,  2 columns: Lunar Phases
     273 rows,  7 columns: Ephemerides
     153 rows, 12 columns: DSSI Observations
      40 rows,  6 columns: SIDE Observations
     130 rows, 11 columns: Speckle Detections
     589 rows, 10 columns: Phase Events
     176 rows, 14 columns: PEPSI Observations



In [14]:
html.clear_directory(html_dir)

altitude_categories = [
    ((-90, min_altitude.value), 0),
    ((min_altitude.value, 90), 1),
]

pl = pr.PriorityList(tl, session, interval=30 * u.min)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.prioritize_phase_sequence(pl, ["Eclipse"], "Eclipse", True, True, True)
# pr.prioritize_side_observation(pl, side_state="Eclipse")
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.0)
pl.categorize_priorities(bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"])

html.render_observing_pages(tl, pl, {}, html_dir)
# pl.categorical_priorities[0]